## **1. Install and import libraries**

In [1]:
!pip install -qq datasets evaluate transformers[sentencepiece]
!pip install -qq accelerate
!apt install git-lfs
!pip install tokenizers==0.21.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.

In [3]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.5 MB/s eta 0:00:00


In [5]:
import os
import math
import torch
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer

import evaluate
import transformers

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

## **2. Download and load poem dataset**

In [12]:
from datasets import Dataset
import pandas as pd

# Load each line of Truyện Kiều as a poem line
with open("truyen_kieu_data.txt", "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# Create dataset (each line as a document)
dataset = Dataset.from_dict({"text": lines})
dataset = dataset.train_test_split(test_size=0.1)


In [13]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
import re

def clean_line(line):
    line = re.sub(r"^\d+[\.\-: ]*", "", line)
    return line.strip()


# Tokenizer setup

UNK_TOKEN = "[UNK]"
PAD_TOKEN = "[PAD]"
EOS_TOKEN = "[EOS]"
SOS_TOKEN = "[SOS]"
EOL_TOKEN = "[EOL]"

tokenizer = Tokenizer(WordLevel(unk_token=UNK_TOKEN))
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(
    special_tokens=[UNK_TOKEN, PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, EOL_TOKEN]
)


# Train tokenizer on cleaned lines

def text_generator():
    with open("truyen_kieu_data.txt", "r", encoding="utf-8") as f:
        for line in f:
            clean = clean_line(line)
            if clean:
                yield clean

tokenizer.train_from_iterator(text_generator(), trainer=trainer)


# Padding & truncation setup

PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)
EOS_TOKEN_ID = tokenizer.token_to_id(EOS_TOKEN)
MAX_SEQ_LEN = 25

tokenizer.enable_padding(length=MAX_SEQ_LEN, pad_token=PAD_TOKEN, pad_id=PAD_TOKEN_ID)
tokenizer.enable_truncation(max_length=MAX_SEQ_LEN)


# Encode & decode a sample line

with open("truyen_kieu_data.txt", "r", encoding="utf-8") as f:
    raw_lines = [line.strip() for line in f if line.strip()]
    sample_line = clean_line(raw_lines[1])

encoded = tokenizer.encode(sample_line)
decoded = tokenizer.decode(encoded.ids)

print("Cleaned line:", sample_line)
print("Token IDs:", encoded.ids)
print("Tokens:", encoded.tokens)
print("Attention Mask:", encoded.attention_mask)
print("Decoded text:", decoded)


Cleaned line: Chữ tài chữ mệnh khéo là ghét nhau.
Token IDs: [556, 130, 170, 412, 1221, 12, 1458, 75, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Tokens: ['Chữ', 'tài', 'chữ', 'mệnh', 'khéo', 'là', 'ghét', 'nhau', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded text: Chữ tài chữ mệnh khéo là ghét nhau .


In [14]:
import re
from typing import List, Tuple
from torch.utils.data import Dataset
def split_poems(text: str) -> List[List[str]]:
    """
    Split text line-by-line into 4-line stanzas (used for Truyện Kiều).
    Ignores blank lines and batches every 4 lines.
    """
    def clean_line(line: str) -> str:
        return re.sub(r"^\d+[\.\-: ]*", "", line).strip()

    lines = [clean_line(line) for line in text.strip().split("\n") if line.strip()]
    stanzas = [lines[i:i+4] for i in range(0, len(lines), 4) if len(lines[i:i+4]) == 4]
    return stanzas


In [15]:
def encode_with_padding(text: str, tokenizer, max_len: int) -> Tuple[List[int], List[int]]:
    """
    Encode input text with padding and truncation.
    Returns token ids and attention mask.
    """
    encoded = tokenizer.encode(text)
    ids = encoded.ids
    mask = encoded.attention_mask

    # Pad or truncate
    if len(ids) < max_len:
        pad_len = max_len - len(ids)
        pad_id = tokenizer.token_to_id("[PAD]")
        ids += [pad_id] * pad_len
        mask += [0] * pad_len
    else:
        ids = ids[:max_len]
        mask = mask[:max_len]
    return ids, mask


In [16]:
def prepare_sample(lines: List[str], tokenizer, max_len: int) -> Tuple[List[int], List[int], List[int]]:
    """
    Convert a 4-line stanza into input, target, and attention mask tensors.
    - Input: first 3 lines joined
    - Target: 4th line
    """
    input_text = "\n".join(lines[:3])
    target_text = lines[3]

    input_ids, input_mask = encode_with_padding(input_text, tokenizer, max_len)
    target_ids, _ = encode_with_padding(target_text, tokenizer, max_len)

    return input_ids, target_ids, input_mask


In [17]:
def build_dataset_from_file(file_path: str, tokenizer, max_len: int):
    with open(file_path, "r", encoding="utf-8") as f:
        raw_text = f.read()

    all_inputs, all_targets, all_masks = [], [], []
    stanzas = split_poems(raw_text)
    print(f"Number of valid stanzas: {len(stanzas)}")

    for stanza in stanzas:
        input_ids, target_ids, mask = prepare_sample(stanza, tokenizer, max_len)
        all_inputs.append(input_ids)
        all_targets.append(target_ids)
        all_masks.append(mask)

    print(f"Number of stanzas processed: {len(all_inputs)}")

    return (
        torch.tensor(all_inputs, dtype=torch.long),
        torch.tensor(all_targets, dtype=torch.long),
        torch.tensor(all_masks, dtype=torch.float),
    )


In [18]:
with open("truyen_kieu_data.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

stanzas = split_poems(raw_text)
print(" Number of valid stanzas:", len(stanzas))

 Number of valid stanzas: 814


In [19]:
class PoemDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_seq_len):
        self.input_seqs, self.target_seqs, self.attn_masks = build_dataset_from_file(
            file_path, tokenizer, max_seq_len
        )

    def __len__(self):
        return len(self.input_seqs)

    def __getitem__(self, idx):
        return self.input_seqs[idx], self.target_seqs[idx], self.attn_masks[idx]


In [20]:
dataset = PoemDataset("truyen_kieu_data.txt", tokenizer=tokenizer, max_seq_len=25)
print("Number of samples:", len(dataset))

# Inspect one training item
x, y, mask = dataset[0]
print("Input IDs:", x)
print("Target IDs:", y)
print("Attention mask:", mask)


Number of valid stanzas: 814
Number of stanzas processed: 814
Number of samples: 814
Input IDs: tensor([ 363,   80,   45,  575,   11,   63,    5,  556,  130,  170,  412, 1221,
          12, 1458,   75,    6, 1813,  199,    8,  965,  203,  724,    5,    1,
           1])
Target IDs: tensor([ 168,  107,   93,   60,   37,  258, 1084,   16,    6,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1])
Attention mask: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0.])


In [21]:
def prepare_sample(tokenizer, stanza: List[str], max_len: int):
    """
    Given a 4-line poem stanza, create training pairs where:
    - input = prefix tokens
    - target = next token
    - attention_mask = marks real vs. padded tokens
    """
    input_seqs = []
    target_seqs = []
    attn_masks = []

    # Build the full poem string with special tokens
    input_text = "[SOS] " + " [EOL] ".join(stanza) + " [EOL] [EOS]"
    full_encoding = tokenizer.encode(input_text)
    input_ids = full_encoding.ids

    for i in range(1, len(input_ids)):
        prefix_ids = input_ids[:i]
        target_ids = input_ids[1:i + 1]

        # Decode and re-encode to get attention mask and padding
        prefix_text = tokenizer.decode(prefix_ids, skip_special_tokens=False)
        target_text = tokenizer.decode(target_ids, skip_special_tokens=False)

        prefix_encoding = tokenizer.encode(prefix_text)
        target_encoding = tokenizer.encode(target_text)

        # Pad/truncate manually
        pad_id = tokenizer.token_to_id("[PAD]")
        if len(prefix_encoding.ids) < max_len:
            diff = max_len - len(prefix_encoding.ids)
            prefix_encoding.ids = prefix_encoding.ids + [pad_id] * diff # Change: Use list concatenation instead of assignment
            prefix_encoding.attention_mask = prefix_encoding.attention_mask + [0] * diff  # Change: Use list concatenation instead of assignment
        else:
            prefix_encoding.ids = prefix_encoding.ids[:max_len] # Change: Assign sliced list to a new variable
            prefix_encoding.attention_mask = prefix_encoding.attention_mask[:max_len]  # Change: Assign sliced list to a new variable

        if len(target_encoding.ids) < max_len:
            target_encoding.ids = target_encoding.ids + [pad_id] * (max_len - len(target_encoding.ids))  # Change: Use list concatenation instead of assignment
        else:
            target_encoding.ids = target_encoding.ids[:max_len] # Change: Assign sliced list to a new variable

        input_seqs.append(prefix_encoding.ids)
        target_seqs.append(target_encoding.ids)
        attn_masks.append(prefix_encoding.attention_mask)

    return input_seqs, target_seqs, attn_masks



In [23]:
def create_samples(file_path, tokenizer, max_len):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    all_inputs, all_targets, all_masks = [], [], []

    stanzas = split_poems(text)
    print(f"Found {len(stanzas)} stanzas")

    for stanza in stanzas:
        inputs, targets, masks = prepare_sample(tokenizer, stanza, max_len)
        all_inputs.extend(inputs)
        all_targets.extend(targets)
        all_masks.extend(masks)

    return (
        torch.tensor(all_inputs, dtype=torch.long),
        torch.tensor(all_targets, dtype=torch.long),
        torch.tensor(all_masks, dtype=torch.float),
    )


In [24]:
class PoemDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, tokenizer, max_len):
        self.inputs, self.targets, self.masks = create_samples(file_path, tokenizer, max_len)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx], self.masks[idx]


In [25]:

import math

class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dim, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Create positional encoding matrix
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-math.log(10000.0) / embedding_dim))
        pe = torch.zeros(max_len, embedding_dim)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # Shape: (1, max_len, embedding_dim)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


In [26]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_heads, hidden_dim, n_layers, dropout=0.1):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_encoder = PositionalEncoding(embedding_dim, dropout=dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=n_heads,
            dim_feedforward=hidden_dim,
            dropout=dropout,
            batch_first=True
        )

        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.decoder = nn.Linear(embedding_dim, vocab_size)
        self.embedding_dim = embedding_dim

        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.decoder.weight)
        self.decoder.bias.data.fill_(0)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        """
        src: (batch_size, seq_len)
        src_mask: (seq_len, seq_len)
        src_key_padding_mask: (batch_size, seq_len)
        """
        x = self.embedding(src) * math.sqrt(self.embedding_dim)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(
            x,
            mask=src_mask,
            src_key_padding_mask=src_key_padding_mask
        )
        logits = self.decoder(x)
        return logits


In [33]:
def beam_search(beams, model, device, beam_width, eos_token_id):
    candidates = []

    for score, seq in beams:
        if seq[-1] == eos_token_id:
            candidates.append((score, seq))
            continue

        input_tensor = torch.tensor([seq], dtype=torch.long).to(device)

        with torch.no_grad():
            output = model(input_tensor)

        log_probs = torch.log_softmax(output[0, -1], dim=-1)
        top_scores, top_indices = torch.topk(log_probs, beam_width)

        for i in range(beam_width):
            next_seq = seq + [top_indices[i].item()]
            new_score = score + top_scores[i].item()
            candidates.append((new_score, next_seq))

    # Keep top-k candidates
    candidates = sorted(candidates, key=lambda x: x[0], reverse=True)
    return candidates[:beam_width]


In [27]:
def greedy_sampling(model, tokenizer, input_text, max_len):
    tokenizer.no_padding()
    tokenizer.no_truncation()

    input_encoded = tokenizer.encode(input_text)
    input_ids = input_encoded.ids
    eos_token_id = tokenizer.token_to_id("[EOS]")

    generated_ids = input_ids.copy()

    model.eval()
    for _ in range(max_len - len(input_ids)):
        input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(model.device)

        with torch.no_grad():
            outputs = model(input_tensor)

        next_token_id = torch.argmax(outputs[0, -1, :]).item()
        generated_ids.append(next_token_id)

        if next_token_id == eos_token_id:
            break

    return tokenizer.decode(generated_ids, skip_special_tokens=False)


In [34]:
def generate_poem_with_sampling(model, tokenizer, prompt, max_len=50, temperature=1.0):
    """
    Generate poem using temperature-controlled sampling.
    """
    tokenizer.no_padding()
    tokenizer.no_truncation()

    model.eval()
    input_ids = tokenizer.encode(prompt).ids
    eos_token_id = tokenizer.token_to_id("[EOS]")

    generated_ids = input_ids.copy()

    for _ in range(max_len - len(input_ids)):
        input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(model.device)

        with torch.no_grad():
            outputs = model(input_tensor)
            last_token_logits = outputs[0, -1, :]  # logits for last token

        next_token_id = sample_with_temperature(last_token_logits, temperature)
        generated_ids.append(next_token_id)

        if next_token_id == eos_token_id:
            break

    return tokenizer.decode(generated_ids, skip_special_tokens=False)


In [35]:
VOCAB_SIZE = len(tokenizer.get_vocab())
EMBEDDING_DIMS = 128
HIDDEN_DIMS = 64
N_LAYERS = 4
N_HEADS = 16
DROPOUT = 0.2
MAX_SEQ_LEN = 25
BATCH_SIZE = 32
LR = 5.0
EPOCHS = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [36]:
model = TransformerModel(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIMS,
    n_heads=N_HEADS,
    hidden_dim=HIDDEN_DIMS,
    n_layers=N_LAYERS,
    dropout=DROPOUT
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)


In [39]:
def format_luc_bat(text: str):
    text = re.sub(r"[^\w\s]", "", text)
    words = text.strip().split()

    lines = []
    i = 0
    toggle = 6
    while i < len(words):
        line = words[i:i+toggle]
        if len(line) < toggle:
            break
        line[0] = line[0].capitalize()
        lines.append(" ".join(line))
        i += toggle
        toggle = 8 if toggle == 6 else 6
    return lines

In [40]:
model.eval()
beam_width = 5
max_len = 50

prompt = "[SOS] Trăng "
tokenizer.no_padding()
tokenizer.no_truncation()

input_ids = tokenizer.encode(prompt).ids
eos_token_id = tokenizer.token_to_id("[EOS]")

beams = [(0.0, input_ids)]
for _ in range(max_len):
    beams = beam_search(beams, model, device, beam_width, eos_token_id)
    if all(seq[-1] == eos_token_id for _, seq in beams):
        break

# Get best result
best_score, best_seq = beams[0]
generated = tokenizer.decode(best_seq, skip_special_tokens=False)

# Clean special tokens
cleaned = generated.replace("[SOS]", "").replace("[EOS]", "").replace("[EOL]", "")

# Format as luc_bat
formatted_lines = format_luc_bat(cleaned)
for line in formatted_lines:
    print(line)


Trăng một một Xem cái há
Một một một một một một một một
Một một một một một một
Một một một một một một một một
Một một một một một một
Một một một một một một một dựng
Một một một một một một


In [41]:
def sample_with_temperature(logits, temperature=1.0):
    """
    Apply temperature scaling to logits before softmax sampling.
    Lower temperature → more deterministic.
    Higher temperature → more diverse output.
    """
    if temperature != 1.0:
        logits = logits / temperature
    probs = F.softmax(logits, dim=-1)
    sampled_id = torch.multinomial(probs, 1).item()
    return sampled_id


In [42]:
def generate_poem_with_sampling(model, tokenizer, prompt, max_len=50, temperature=1.0, device='cuda'):
    """
    Generate poem using temperature-controlled sampling.
    """
    tokenizer.no_padding()
    tokenizer.no_truncation()

    model.eval()
    input_ids = tokenizer.encode(prompt).ids
    eos_token_id = tokenizer.token_to_id("[EOS]")

    generated_ids = input_ids.copy()

    for _ in range(max_len - len(input_ids)):
        input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(device) # Pass the device to the .to() method

        with torch.no_grad():
            outputs = model(input_tensor)
            last_token_logits = outputs[0, -1, :]  # (vocab_size)

        next_token_id = sample_with_temperature(last_token_logits, temperature)
        generated_ids.append(next_token_id)

        if next_token_id == eos_token_id:
            break

    return tokenizer.decode(generated_ids, skip_special_tokens=False)

In [50]:
def format_luc_bat(text: str):
    text = re.sub(r"[^\w\sàáảãạăằắẳẵặâầấẩẫậèéẻẽẹêềếểễệìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵ]", "", text)
    words = text.strip().split()
    lines = []
    i = 0
    toggle = 6
    while i + toggle <= len(words):
        line = words[i:i+toggle]
        line[0] = line[0].capitalize()
        line_text = " ".join(line)
        if toggle == 8 and not check_luc_bat_tone_rule(line_text):
            i += 1
            continue
        lines.append(line_text)
        i += toggle
        toggle = 8 if toggle == 6 else 6
    return lines


In [53]:
prompt = "[SOS] quang "



temperature = 1.2
generated = generate_poem_with_sampling(model, tokenizer, prompt, max_len=50, temperature=temperature, device='cpu')

# Clean tokens
cleaned = generated.replace("[SOS]", "").replace("[EOS]", "").replace("[EOL]", "")


for line in format_luc_bat(cleaned):
    print(line)


Quang chẳng Vâng giáp chiêng Chiều
Thèm Cớ nham nói đứng diệp bình đức
Dày giếng ông ả nhẫn dọn
Thui ngờ thốt cái Mập ông bảnh nghĩa


Training on GPT2 model

In [54]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
import re

#apply the lowercase this time
def clean_line(line):
    line = re.sub(r"^\d+[\.\-: ]*", "", line)
    return line.strip().lower()


# Tokenizer setup

UNK_TOKEN = "[UNK]"
PAD_TOKEN = "[PAD]"
EOS_TOKEN = "[EOS]"
SOS_TOKEN = "[SOS]"
EOL_TOKEN = "[EOL]"

tokenizer = Tokenizer(WordLevel(unk_token=UNK_TOKEN))
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(
    special_tokens=[UNK_TOKEN, PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, EOL_TOKEN]
)


# Train tokenizer on cleaned lines

def text_generator():
    with open("truyen_kieu_data.txt", "r", encoding="utf-8") as f:
        for line in f:
            clean = clean_line(line)
            if clean:
                yield clean

tokenizer.train_from_iterator(text_generator(), trainer=trainer)


# Padding & truncation setup

PAD_TOKEN_ID = tokenizer.token_to_id(PAD_TOKEN)
EOS_TOKEN_ID = tokenizer.token_to_id(EOS_TOKEN)
MAX_SEQ_LEN = 25

tokenizer.enable_padding(length=MAX_SEQ_LEN, pad_token=PAD_TOKEN, pad_id=PAD_TOKEN_ID)
tokenizer.enable_truncation(max_length=MAX_SEQ_LEN)


# Encode & decode a sample line

with open("truyen_kieu_data.txt", "r", encoding="utf-8") as f:
    raw_lines = [line.strip() for line in f if line.strip()]
    sample_line = clean_line(raw_lines[1])


encoded = tokenizer.encode(sample_line)
decoded = tokenizer.decode(encoded.ids)

print("Cleaned line:", sample_line)
print("Token IDs:", encoded.ids)
print("Tokens:", encoded.tokens)
print("Attention Mask:", encoded.attention_mask)
print("Decoded text:", decoded)


Cleaned line: chữ tài chữ mệnh khéo là ghét nhau.
Token IDs: [136, 134, 136, 396, 369, 17, 1227, 93, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Tokens: ['chữ', 'tài', 'chữ', 'mệnh', 'khéo', 'là', 'ghét', 'nhau', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded text: chữ tài chữ mệnh khéo là ghét nhau .


In [56]:
from datasets import Dataset

with open("truyen_kieu_data.txt", "r", encoding="utf-8") as f:
    lines = [line.strip().lower() for line in f if line.strip()]
dataset = Dataset.from_dict({"text": lines})

# Optional: split into train/test
dataset = dataset.train_test_split(test_size=0.1)


In [57]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load pretrained Vietnamese GPT-2
model = AutoModelForCausalLM.from_pretrained("danghuy1999/gpt2-viwiki")
tokenizer = AutoTokenizer.from_pretrained("danghuy1999/gpt2-viwiki", use_fast=False)

# Set pad token
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/773k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

In [58]:
# Tokenize and add labels for causal LM
def tokenize(batch):
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=64)
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/2932 [00:00<?, ? examples/s]

Map:   0%|          | 0/326 [00:00<?, ? examples/s]

In [ ]:
# Provide a verse or phrase to start
input_text = "trăm năm trong cõi người ta,\n"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Move input_ids to the same device as the model
# Assuming your model is on cuda:0
input_ids = input_ids.to('cuda') # or input_ids.to(model.device) if you are unsure


# Generate continuation
output = model.generate(
    input_ids,
    max_length=40,
    temperature=0.95,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
    do_sample=True
)

# Decode and print
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


trăm năm trong cõi người ta,



In [ ]:
MAX_LEN = 64

def tokenize(batch):
    inputs = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN
    )
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/2931 [00:00<?, ? examples/s]

Map:   0%|          | 0/326 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./gpt2_kieu_model",
    # Replace 'evaluation_strategy' with 'eval_strategy'
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

<ipython-input-178-9aeafd9022fc>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,6.460541
2,5.948500,6.272570
3,5.273300,6.207601
4,5.273300,6.203349
5,4.970300,6.212679


TrainOutput(global_step=1835, training_loss=5.284172037316928, metrics={'train_runtime': 367.7977, 'train_samples_per_second': 39.845, 'train_steps_per_second': 4.989, 'total_flos': 478654341120000.0, 'train_loss': 5.284172037316928, 'epoch': 5.0})

In [ ]:
import torch
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [62]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/gpt2_kieu_model"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_path)
model.to("cuda" if torch.cuda.is_available() else "cpu")


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [122]:
def format_poem(poem):
    lines = poem.strip().split()
    verses = []
    current = []

    for word in lines:
        current.append(word)
        if len(current) >= 6 and (len(verses) % 2 == 0 and len(current) == 6) or \
           (len(verses) % 2 == 1 and len(current) == 8):
            verses.append(" ".join(current))
            current = []

    return "\n".join(verses)


In [136]:
import unicodedata

def get_tone_class(syllable):
    syllable = unicodedata.normalize('NFC', syllable.lower())
    for char in syllable[::-1]:
        if char in 'àèìòùỳ':
            return 'bằng'
        elif char in 'áéíóúýảẻỉỏủỷãẽĩõũỹạẹịọụỵ':
            return 'trắc'
    return 'bằng'


In [137]:
def check_luc_bat_tone_rule(line):
    words = line.strip().split()
    if len(words) != 8:
        return False
    expected = ['bằng', 'trắc', 'bằng', 'bằng']
    positions = [1, 3, 5, 7]  # 0-based

    for idx, exp in zip(positions, expected):
        tone = get_tone_class(words[idx])
        if tone != exp:
            return False
    return True


In [138]:
def generate_line(prompt, target_syllables, max_new_tokens=12, max_attempts=10):
    for _ in range(max_attempts):
        inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=64)
        input_ids = inputs['input_ids'].to(model.device)
        attention_mask = inputs['attention_mask'].to(model.device)

        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            temperature=0.8,
            top_k=40,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.pad_token_id
        )

        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        new_line = decoded[len(prompt):].strip()
        if count_syllables(new_line) != target_syllables:
            continue
        if target_syllables == 8 and not check_luc_bat_tone_rule(new_line):
            continue
        return new_line
    return "[FAILED TO GENERATE LINE]"


In [139]:
input_text =  "thương sao cho trọn thì thương"

In [140]:


# Tokenize
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=64,
)

input_ids = inputs["input_ids"].to(model.device)
attention_mask = inputs["attention_mask"].to(model.device)

# Generate
output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=20,
    temperature=0.8,
    top_k=40,
    top_p=0.9,
    repetition_penalty=1.2,
    no_repeat_ngram_size=2,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id
)

# Decode
generated = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated poem:\n", generated)


Generated poem:
 thương sao cho trọn thì thương lòng, chẳng thương cả hai bên kia, nên ai cũng thương lòng này là người này hay không


In [150]:
import unicodedata

def generate_luc_bat_poem_gpt2(model, tokenizer, prompt, max_lines=3):
    def get_tone_class(syllable):
        """
        Phân loại thanh của một âm tiết thành 'bằng' hoặc 'trắc'.
        - Bằng: thanh ngang (không dấu), huyền
        - Trắc: sắc, hỏi, ngã, nặng
        """
        syllable = unicodedata.normalize('NFC', syllable.lower())
        for char in syllable[::-1]:
            if char in 'àèìòùỳ':
                return 'bằng'
            elif char in 'áéíóúýảẻỉỏủỷãẽĩõũỹạẹịọụỵ':
                return 'trắc'
        return 'bằng'

    def check_luc_bat_tone_rule(line):
        words = line.strip().split()
        if len(words) != 8:
            return False
        expected = ['bằng', 'trắc', 'bằng', 'bằng']
        positions = [1, 3, 5, 7]
        return all(get_tone_class(words[idx]) == exp for idx, exp in zip(positions, expected))

    def split_luc_bat_poem(raw_text):
        words = raw_text.strip().split()
        lines, i, toggle = [], 0, 6
        while i + toggle <= len(words) and len(lines) < max_lines:
            line = " ".join(words[i:i+toggle])
            if toggle == 8 and not check_luc_bat_tone_rule(line):
                i += 1
                continue
            lines.append(line)
            i += toggle
            toggle = 8 if toggle == 6 else 6
        return lines

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        temperature=0.9,
        top_k=40,
        top_p=0.95,
        repetition_penalty=1.1,
        no_repeat_ngram_size=2,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    cleaned = generated.replace("[SOS]", "").replace("[EOS]", "").replace("[EOL]", "")

    return split_luc_bat_poem(cleaned)


In [151]:
def generate_luc_bat_poem_retry(model, tokenizer, prompt, max_lines=3, max_attempts=10):
    for attempt in range(max_attempts):
        lines = generate_luc_bat_poem_gpt2(model, tokenizer, prompt, max_lines=max_lines)
        if len(lines) == max_lines:
            return lines
    return ["[FAILED]"] * max_lines


In [152]:
prompt = generate_luc_bat_poem_gpt2(model, tokenizer, "thương sao cho trọn thì thương")
print("Lục bát poem:")
for line in prompt:
    print(line)


Lục bát poem:
thương sao cho trọn thì thương
thương nàng, rồi lại dò lời, sau này
là người con đi: ta, con


In [132]:
from unicodedata import normalize

def get_tone_class(syllable):
    syllable = normalize('NFC', syllable.lower())
    for char in syllable[::-1]:
        if char in 'àèìòùỳảẻỉỏủỷãẽĩõũỹạẹịọụỵ':
            return 'bằng'
        elif char in 'áéíóúý':
            return 'trắc'
    return 'bằng'

def check_luc_bat_tone_rule(line):
    words = line.strip().split()
    if len(words) != 8:
        return False
    expected = ['bằng', 'trắc', 'bằng', 'bằng']
    positions = [1, 3, 5, 7]
    return all(get_tone_class(words[idx]) == exp for idx, exp in zip(positions, expected))

def split_luc_bat_poem_debug(raw_text, max_lines=3):
    words = raw_text.strip().split()
    lines, i, toggle = [], 0, 6
    while i + toggle <= len(words) and len(lines) < max_lines:
        line = " ".join(words[i:i+toggle])
        if toggle == 8 and not check_luc_bat_tone_rule(line):
            print(f" Rejected (tone rule): {line}")
            i += 1
            continue
        if len(line.split()) != toggle:
            print(f" Rejected (wrong syllables): {line}")
            i += 1
            continue
        print(f" Accepted: {line}")
        lines.append(line)
        i += toggle
        toggle = 8 if toggle == 6 else 6
    return lines


In [135]:
def generate_luc_bat_poem_retry_debug(model, tokenizer, prompt, max_lines=3, max_attempts=10):
    for attempt in range(1, max_attempts + 1):
        print(f"\n Attempt {attempt}/{max_attempts}")
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        output = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            temperature=0.9,
            top_k=40,
            top_p=0.95,
            repetition_penalty=1.1,
            no_repeat_ngram_size=2,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )

        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        cleaned = decoded.replace("[SOS]", "").replace("[EOS]", "").replace("[EOL]", "")

        print("Raw:", cleaned)
        lines = split_luc_bat_poem_debug(cleaned, max_lines=max_lines)

        if len(lines) == max_lines:
            print("Success!")
            return lines

    print(" Failed after all attempts.")
    return ["[Failed]"] * max_lines


In [134]:
poem = generate_luc_bat_poem_retry_debug(model, tokenizer, "thương sao cho trọn thì thương")
for line in poem:
    print(line)



 Attempt 1/10
 Raw: thương sao cho trọn thì thương này ta đâu đây, thương ai mà ở trên dưới này, đây xưa nay cho đây nay, xưa kia anh ta, người thân xưa sau xưa, sao ở đây xa xưa đời đời xưa nhất: sao đâu này ở đó, thiên này nào
 Accepted: thương sao cho trọn thì thương
 Rejected (tone rule): này ta đâu đây, thương ai mà ở
 Rejected (tone rule): ta đâu đây, thương ai mà ở trên
 Rejected (tone rule): đâu đây, thương ai mà ở trên dưới
 Rejected (tone rule): đây, thương ai mà ở trên dưới này,
 Rejected (tone rule): thương ai mà ở trên dưới này, đây
 Rejected (tone rule): ai mà ở trên dưới này, đây xưa
 Rejected (tone rule): mà ở trên dưới này, đây xưa nay
 Rejected (tone rule): ở trên dưới này, đây xưa nay cho
 Rejected (tone rule): trên dưới này, đây xưa nay cho đây
 Rejected (tone rule): dưới này, đây xưa nay cho đây nay,
 Rejected (tone rule): này, đây xưa nay cho đây nay, xưa
 Rejected (tone rule): đây xưa nay cho đây nay, xưa kia
 Rejected (tone rule): xưa nay cho đây nay, xưa kia a

In [158]:
#Adjust the tone

def get_tone_class(syllable):


    syllable = unicodedata.normalize('NFC', syllable.lower())
    for char in syllable[::-1]:
        if char in 'àèìòùỳ':  # huyền → bằng
            return 'bằng'
        elif char in 'áéíóúýảẻỉỏủỷãẽĩõũỹạẹịọụỵ':  # all trắc tones
            return 'trắc'
    return 'bằng'  # no diacritic → thanh ngang → bằng

def check_luc_bat_tone_rule(line):

    words = line.strip().split()
    if len(words) != 8:
        return False
    expected = ['bằng', 'trắc', 'bằng', 'bằng']
    positions = [1, 3, 5, 7]
    return all(get_tone_class(words[idx]) == exp for idx, exp in zip(positions, expected))

def split_luc_bat_poem_debug(raw_text, max_lines=3):

    words = raw_text.strip().split()
    lines, i, toggle = [], 0, 6

    while i + toggle <= len(words) and len(lines) < max_lines:
        line = " ".join(words[i:i + toggle])

        if toggle == 8 and not check_luc_bat_tone_rule(line):
            print(f"Rejected (tone rule): {line}")
            i += 1
            continue

        if len(line.split()) != toggle:
            print(f"Rejected (wrong syllables): {line}")
            i += 1
            continue

        print(f"Accepted: {line}")
        lines.append(line)
        i += toggle
        toggle = 8 if toggle == 6 else 6

    return lines


In [156]:
prompt = "thương sao cho trọn thì thương"
generated_lines = generate_luc_bat_poem_retry(model, tokenizer, prompt)

for line in generated_lines:
    print(line)


thương sao cho trọn thì thương
xót cho nàng cũng hay, đây cũng là
thương, thương thương hay thương thường


Test with different style

In [159]:
prompt = "deadline chưa nộp vẫn chill"
poem = generate_luc_bat_poem_retry(model, tokenizer, prompt)
for line in poem:
    print(line)


deadline chưa nộp vẫn chill, chặt
ai biết thương xót này, ai ai cho
thương xa: ai cũng có thương:


In [160]:
prompt = "trả bài chưa nhớ tên môn"
poem = generate_luc_bat_poem_retry(model, tokenizer, prompt)
for line in poem:
    print(line)

trả bài chưa nhớ tên môn,
lại nhà, ai cũng đành sang ta, ta
này người ai cho ai, vì
